# Drought assessment and monitoring 


* **Special requirements:** A Google account, access to Google Earth Engine; 
* **Prerequisites:** lecture of the week

## Background
To help with **drought** classification and monitoring, scientists have defined several types of drought. **Meteorological drought** is based on the degree of dryness or rainfall deficit and the length of the dry period. **Hydrological drought** is based on the impact of rainfall deficits on the water supply such as stream flow, reservoir and lake levels, and ground water table decline. **Agricultural drought** is when crops become affected by drought.

**Palmer Drought Severity Index (PDSI)** uses readily available temperature and precipitation data to estimate relative dryness. It is a standardized index that generally spans -10 (dry) to +10 (wet). Read more about PDSI [here](https://journals.ametsoc.org/view/journals/apme/23/7/1520-0450_1984_023_1100_tpdsil_2_0_co_2.xml). It is a commonly used meteorological drought indicator.

The **GRACE** twin satellite was launched in 2002 to map the Earth's gravity field changes. The observed monthly changes in gravity are caused by monthly changes in mass. The mass changes can be thought of as concentrated in a very thin layer of water thickness changes near the Earth's surface. In reality, most of the monthly gravity changes are indeed caused by changes in water storage in hydrologic reservoirs, by moving ocean, atmospheric and land ice masses, and by mass exchanges between these Earth system compartments. Therefore, the change over the land is mainly caused by the change in total water storage including change in snow, surface water, soil water, groundwater and water in the vegetation. The GRACE-based drought indicator is a indicator of hydrological drought.   

## Aims of the practical session

1. Explore the commonly used drought index, PDSI, to identify the drought events in Australia
2. Understand the observation of total water storage 
3. Understand different drought indices 

## Description

1. Firstly, we explore the PDSI data and extract the values for a basin and identify dry and wet periods from the PDSI
2. Secondly, we explore the GRACE total water storage observations and calculate GRACE-based drought index (GRACE-DSI)
3. Finally we compare the difference between PDSI and GRACE-DSI

***

## Load packages
Import Python packages that are used for the analysis.

Use standard import commands; some are shown below. 
Begin with any `iPython` magic commands, followed by standard Python packages, then any additional functionality you need from the `Scripts` directory.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import geemap as gmap
import ee
import geemap.colormaps as cm

### Connect to Google Earth Engine (GEE)

Connect to the GEE so we can access GEE datasets and computing assets.

## Part 1 - Explore PDSI for meteorological drought

The PDSI product we are going to use is the [TerraClimate](https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_TERRACLIMATE?hl=en#description). It is a dataset of monthly climate and climatic water balance for global terrestial surface. 


In [ ]:
# get the Palmer Drought Severity Index (PDSI) from the TerraClimate dataset for 2002 to 2016
Map = gmap.Map(center=[-31,150], zoom=5)
Map.addLayerControl()

PDSI = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').filter(ee.Filter.date('2002-01-01', '2017-01-01')).select('pdsi')
PDSIimages = PDSI.toBands().multiply(0.01); #0.01 is the scale factor 

vis_params = {
  'min':-10,
  'max': 10,
  'palette': cm.palettes.RdBu
};

Map.add_time_slider(PDSIimages, vis_params,  layer_name="PDSI")

Map.add_colorbar_branca(colors=vis_params['palette'],
                        vmin=vis_params['min'],
                        vmax=vis_params['max'],
                        layer_name="PDSI")
Map


#### Use the plotting tool to plot the PDSI values for a point or a region in Australia

Are there more dry or wet years in your time series plot?

### Let's extract the PDSI values for a basin.

In [ ]:
# we get the basin boundary from the Feature collection of global Basins 
basins = ee.FeatureCollection('WWF/HydroSHEDS/v1/Basins/hybas_3')
Map.addLayer(basins, {}, 'Basins')
Map


### Select one Basin in Australia for this exercise

Here we can use the Murray-Darling Basin as the example. The HYBAS_ID is 5030073410

In [ ]:
# select the feature by filtering with HYBAS_ID
mdb = ee.FeatureCollection('WWF/HydroSHEDS/v1/Basins/hybas_3').filter(ee.Filter.eq("HYBAS_ID", 5030073410))

Map.addLayer(mdb, {}, 'Murray-Darling Basin')
props = mdb.first().toDictionary().getInfo()
print(props)
Map

<div class="alert alert-block alert-danger"> 

## Exercise 1: Identify the drought events from 2002 to 2016

1. Extract the average (zonal average of all pixels in the basin) PDSI value for Murray-Darling basin in Australia and plot the time series. Most of the code is written for you, but you need to complete the `poi_extract` function by writing a line of code that returns the mean of all pixels in the basin at each time-step.

2. Identify the driest and wettest periods from the time series,

</div>  

<!-- img.reduceRegion(reducer=ee.Reducer.mean(), geometry=mdb).get('pdsi') -->

In [ ]:
# Here is an example of extracting the PDSI value for the Murray-Darling Basin

# here we define a function to extract the value for a region or a point
def poi_extract(img):
    #mean = # ADD YOUR REDUCING CODE HERE
    return img.set('date', img.date().format()).set('PDSI', mean) 

poi_reduced_imgs = PDSI.map(poi_extract)

# create a dataframe for the PDSI values
nested_list = poi_reduced_imgs.reduceColumns(ee.Reducer.toList(2), ['date','PDSI']).values().get(0)
df_pdsi = pd.DataFrame(nested_list.getInfo(), columns=['date','PDSI'])
df_pdsi.PDSI = df_pdsi.PDSI*0.01 # 0.01 is the scale factor
dates_pdsi = df_pdsi['date']
df_pdsi['date'] = pd.to_datetime(dates_pdsi).dt.date #This step is to keep only the date in the index for plotting

# plot the time series
df_pdsi.plot(x='date',y = 'PDSI',ylabel='PDSI',figsize=(15,5))
plt.axhline(y=0, color='grey', linestyle='--')


In [ ]:
# Can you identify driest and wettest period from the time sereis?

# Answer: 

## Explore GRACE observations for hydrological drought monitoring
The product we are going to use is the monthly CSR spherical harmonic solution. Details of the product can be found [here](https://developers.google.com/earth-engine/datasets/catalog/NASA_GRACE_MASS_GRIDS_LAND?hl=en#description). We are going to use the 'lwe_thickness_csr' band

In [ ]:
# Let's have a look at GRACE observed total water storage
Map2 = gmap.Map(center=[-31,150], zoom=4)
GRACE = ee.ImageCollection('NASA/GRACE/MASS_GRIDS/LAND').filter(ee.Filter.date('2002-01-01', '2017-01-01')).select('lwe_thickness_csr');

vis_params = {
  'min':-10,
  'max': 10,
  'palette': cm.palettes.RdBu
};

# we add the surface soil moisture data as a new 'layer' in our map.
Map2.addLayer(GRACE, vis_params, 'GRACE-TWS', True, 1)
Map2.add_colorbar_branca(colors=vis_params['palette'],
                        vmin=vis_params['min'],
                        vmax=vis_params['max'],
                        layer_name="GRACE-TWS")
Map2

<div class="alert alert-block alert-danger"> 

## Exersice 2 Drought index calculation

1. Plot the average total water storage (TWS) anomaly observed by GRACE for the Murray darling basin.

2. Identify dry years from the time series.

3. Calculate the GRACE-based drought index using the formula below. More information about the index can be found [here](https://www.ess.uci.edu/~velicogna/drought_data.html)

    $GRACE-DSI_{i,j} = \frac{TWS_{i,j}-\overline{TWS_j}}{\sigma_j}$

where *i* is the year, *j* is the month ranging from January to December, $\overline{TWS_j}$ and $\sigma_j$ are the mean and standard deviation of TWS anomalies in month *j*.

4. Compare GRACE-DSI with the PDSI and explain the possible reasons for the difference.
    
</div>  


In [ ]:
# Q2.1 Can you plot the average total water storage (TWS) anomaly observed by GRACE for the Murray darling basin?

# Extract the GRACE TWS values for basin average, you can follow the previous example
# CALL THE DATAFRAME "df_grace"

# Your code here 



In [ ]:
# Q2.2 Can you identify dry and wet years from this time series?

# Answer: 

### Function for calculating standardised anomalies

In [ ]:
#function for calculating standarised anomalies (using xarray)
def standardised_anomalies(ds, rename_dict={}):
    '''
    Computes standaradised anomalies on xarray datasets.
    Assumes "ds" has a 'time' coordinate.
    
    Will return an opininated dataframe so will only work
    on the GRACE/PDSI data used for this example.

    Use the 'rename_dict' parameter to change the 
    name of the variable on the dataframe
    '''
    
    #Calculate the climatologies
    clim_mean = ds.groupby('time.month').mean()
    clim_std = ds.groupby('time.month').std()

    #calculate the anomaly
    std_anom = xr.apply_ufunc(lambda x, m, s: (x - m) / s,
                              ds.groupby("time.month"), clim_mean, clim_std)
    
    #tidy up and return dataframe
    std_anom = std_anom.rename(rename_dict).drop('month').to_dataframe()
    
    return std_anom

In [ ]:
# Q2.3 Can you calculate the GRACE-based drought index? 

#First convert GRACE dataset to an xarray object (for easier anomaly calculation - it then works with our function above)
df_grace['date'] = pd.to_datetime(df_grace['date'])
ds_grace = df_grace.rename({'date':'time'},axis=1).set_index('time').to_xarray()

In [ ]:
# Can you use the function provided above to get the GRACE DSI and plot the results?
# YOU WILL NEED TO PROVIDE A RENAMING DICTIONARY - CHANGE THE 'GRACE-TWS' NAME TO 'GRACE-DSI'

# Your code here
grace_std_anom = #CALL FUNCTION HERE

# add some plotting code below


Check the values and categories of GRACE-DSI and identify the driest period. What is the category of this event?

<Img src='https://www.ess.uci.edu/~velicogna/images/table1.jpeg' width=500>

In [ ]:
# Q2.4 Compare GRACE-DSI with the PDSI 

# Calculate standarised anomalies for the PDSI to be comparable to GRACE-DSI using the same function

#convert to xarray
df_pdsi['date'] = pd.to_datetime(df_pdsi['date'])
ds_pdsi = df_pdsi.rename({'date':'time'},axis=1).set_index('time').to_xarray()

#calculate the anomalies, providing a renaming dict, call it 'PDSI-Z'
pdsi_std_anom = #ADD YOUR CODE HERE

# plot two time series together
fig, ax = plt.subplots(figsize=(15,5))
pdsi_std_anom.plot(ax=ax, label='PDSI-Z')
grace_std_anom.plot(ax=ax, label='GRACE-DSI')
ax.legend()
ax.set_ylabel('Standardized Anomalies')
ax.axhline(0, color='grey', linestyle='--')

In [ ]:
# Can you identify dry and wet periods consistently from those two time series? Can you list one possible reason for the difference?

# Answer: 

 

## References

This is where the references go. For exmaple:
- Alley, W.M., 1984: The Palmer Drought Severity Index: limitations and assumptions. Journal of Climate and Applied Meteorology, 23: 1100–1109. DOI: 10.1175/1520-0450(1984)023<1100:TPDSIL>2.0.CO;2.
- Zhao, M., G. A., I. Velicogna, and J. S. Kimball, 2017: A global gridded dataset of GRACE drought severity index for 2002-2014: Comparison with PDSI and SPEI and a case study of the Australia Millennium drought. J. Hydrometeorol., 18, 2117-2129, DOI: https://doi.org/10.1175/JHM-D-16-0182.1
- Wu, Q., (2020). geemap: A Python package for interactive mapping with Google Earth Engine. The Journal of Open Source Software, 5(51), 2305. https://doi.org/10.21105/joss.02305

***

## Additional information

**License:** The code in this notebook was initially created by Siyuan Tian. The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 

**Contact:** If you need assistance, please post a question on the ENGN3903 Wattle (**check**) site 

**Last modified:** August 2023